# Imports and fundamentals

In [1]:
from math import log2, ceil
from typing import List, Set, Tuple

from microschc.binary import Buffer
from microschc.protocol.registry import factory, Stack, PacketParser
from microschc.rfc8724 import PacketDescriptor

from tools.dataset import packets_list
from tools.template import find_templates, Template

all_packets: List[Buffer] = packets_list("./dataset/leshan-thermostat-readings.pcapng")

packets_parser_obj: PacketParser = factory(Stack.IPV6_UDP_COAP)
packets_descriptors: List[PacketDescriptor] = [packets_parser_obj.parse(pkt) for pkt in all_packets]

all_templates: List[Template] = find_templates(all_packets, packets_parser_obj)

In [2]:
from tools.field import fields_as_asciitable

# Filter template's fields from constant and variable
def filter_fields(template: Template) -> List[Tuple[int, str, Set[Buffer]]]:
    template_fields_values: List[Tuple[int, str, Set[Buffer]]] = [(i, field.id.value, set()) for i, field in enumerate(template.fields)]

    for pkt in template.packets:
        for i, field in enumerate(pkt.fields):
            template_fields_values[i][-1].add(field.value)

    return template_fields_values

def display_filtered_fields(template_fields_values: List[Tuple[int, str, Set[Buffer]]]) -> None:
    print(f"Constant fields :\n{fields_as_asciitable(list(filter(lambda t: len(t[-1]) == 1, template_fields_values)))}")
    print(f"Variable fields :\n{fields_as_asciitable(list(filter(lambda t: len(t[-1]) > 1, template_fields_values)))}\n")
    return None

In [3]:
display_filtered_fields(template_fields_values=filter_fields(template=all_templates[0]))

Constant fields :
+------------------------------+-------------------------------------------+
|           field ID           |                   value                   |
+==============================+===========================================+
| 0 - IPv6:Version             | {[----0110 ](4)}                          |
+------------------------------+-------------------------------------------+
| 1 - IPv6:Traffic Class       | {[00000000](8)}                           |
+------------------------------+-------------------------------------------+
| 2 - IPv6:Flow Label          | {[0ff85f](20)}                            |
+------------------------------+-------------------------------------------+
| 4 - IPv6:Next Header         | {[00010001](8)}                           |
+------------------------------+-------------------------------------------+
| 5 - IPv6:Hop Limit           | {[01000000](8)}                           |
+------------------------------+--------------------------

### Rule from template function

In [4]:
from os.path import commonprefix

from tools.rule import rule_descriptor_as_asciitable

from microschc.rfc8724 import RuleDescriptor, RuleFieldDescriptor, Mapping, MatchMapping, DI, MO, CDA
from microschc.binary.buffer import Padding

def create_compression_rule_template(template: Template, rule_ID: bytes, mapping_indexes: List[int] = [], msb_lsb_indexes: List[int] = [], rule_length: int = 4, debug: bool = False) -> RuleDescriptor:
    template_fields_values: List[Tuple[int, str, Set[Buffer]]] = filter_fields(template=template)

    rule_details_by_field: List[RuleFieldDescriptor] = []
    for field_info in template_fields_values:
        field_index, field_id, field_buffer_set = field_info
        
        # Constant
        if len(field_buffer_set) == 1:
            field_value: Buffer = field_buffer_set.pop()
            rule_field_descriptor: RuleFieldDescriptor = RuleFieldDescriptor(
                id=field_id, 
                length=field_value.length, 
                position=0,
                direction=DI.BIDIRECTIONAL, 
                target_value=field_value, 
                matching_operator=MO.EQUAL,
                compression_decompression_action=CDA.NOT_SENT
            )
            
        # Variable
        else:
            if debug:
                print(f"DEBUG : {field_index} : {field_id} -> Card = {len(field_buffer_set)}\n{field_buffer_set}\n")
            
            # Check if (IPv6 or UDP) Field Length or UDP Checksum
            if field_index in [3, 10, 11]:
                rule_field_descriptor: RuleFieldDescriptor = RuleFieldDescriptor(
                    id=field_id, 
                    length=0, 
                    position=0,
                    direction=DI.BIDIRECTIONAL, 
                    target_value=Buffer(content=b'', length=0, padding=Padding.LEFT), 
                    matching_operator=MO.IGNORE,
                    compression_decompression_action=CDA.COMPUTE
                )

            # MSB / LSB
            elif field_index in msb_lsb_indexes:
                hex_contents: List[str] = [hex(field_buffer.value())[2:] for field_buffer in list(field_buffer_set)]

                if len(common_part := commonprefix(hex_contents)) % 2 != 0:
                    common_part = common_part[:-1]

                rule_field_descriptor: RuleFieldDescriptor = RuleFieldDescriptor(
                    id=field_id, 
                    length=list(field_buffer_set)[0].length, 
                    position=0,
                    direction=DI.BIDIRECTIONAL, 
                    target_value=Buffer(content=bytes.fromhex(common_part), length=8*len(bytes.fromhex(common_part))),
                    matching_operator=MO.MSB,
                    compression_decompression_action=CDA.LSB
                )
            
            # Match-mapping / Mapping-sent
            elif field_index in mapping_indexes:
                maximum_field_bits: int = ceil(log2(len(field_buffer_set)))
                content_bytes: bytes = b"\x00"
                bytes_mapping: Mapping = {}
                
                for value in list(field_buffer_set):
                    _tmp_c_b: int = int.from_bytes(content_bytes, byteorder="big") + 1
                    content_bytes = _tmp_c_b.to_bytes(len(content_bytes), byteorder="big")
                    bytes_mapping[value] = Buffer(content=content_bytes, length=maximum_field_bits)

                rule_field_descriptor: RuleFieldDescriptor = RuleFieldDescriptor(
                    id=field_id, 
                    length=list(field_buffer_set)[0].length, 
                    position=0,
                    direction=DI.BIDIRECTIONAL, 
                    target_value=MatchMapping(forward_mapping=bytes_mapping), 
                    matching_operator=MO.MATCH_MAPPING,
                    compression_decompression_action=CDA.MAPPING_SENT
                )

            else:
                rule_field_descriptor: RuleFieldDescriptor = RuleFieldDescriptor(
                    id=field_id, 
                    length=0, 
                    position=0,
                    direction=DI.BIDIRECTIONAL, 
                    target_value=Buffer(content=b'', length=0, padding=Padding.LEFT), 
                    matching_operator=MO.IGNORE,
                    compression_decompression_action=CDA.VALUE_SENT
                )
        
        rule_details_by_field.append(rule_field_descriptor)

    rule: RuleDescriptor = RuleDescriptor(id=Buffer(content=rule_ID, length=rule_length), field_descriptors=rule_details_by_field)

    if debug:
        print(rule_descriptor_as_asciitable(rule))
    
    return rule

### Template 0

In [5]:
""" 
Create compression rule for template 0

Description :
 - 2001:db8::3 -> 2001:db8::20 
 - CoAP non-confirmable and confirmable (2.00 and 2.01), 
 - LwM2M notifications (resource ID 5700)
"""

rule_template_0: RuleDescriptor = create_compression_rule_template(
    template=all_templates[0], 
    rule_ID=b"\x00",
    mapping_indexes=[13, 17, 19, 22, 23], 
    msb_lsb_indexes=[16],   # With 20 -> hits 8334 / 8543 and CF = 4.462 
                            # Without 20 -> hits 8543 / 8543 and CF = 4.435
    debug=False
)

### Template 1

In [6]:
""" 
Create compression rule for template 1

Description :
 - 2001:db8::3 -> 2001:db8::20
 - CoAP non-confirmable and confirmable (2.00 and 2,91)
 - CBOR notifications
 - token 1f0a
"""

rule_template_1: RuleDescriptor = create_compression_rule_template(
    template=all_templates[1], 
    rule_ID=b"\x01", 
    mapping_indexes=[2, 6, 7, 8, 9],
    msb_lsb_indexes=[16, 17], 
    debug=False
)

### Template 2

In [7]:
""" 
Create compression rule for template 2

Description :
 - 2001:db8::20 -> 2001:db8::3
 - ACK (no CoAP Token)
"""

rule_template_2: RuleDescriptor = create_compression_rule_template(
    template=all_templates[2], 
    rule_ID=b"\x02",
    mapping_indexes=[20],
    msb_lsb_indexes=[16, 17], 
    debug=False
)

### Template 3

In [8]:
""" 
Create compression rule for template 3

Description :
 - 2001:db8::20 -> 2001:db8::3 
 - CoAP POST --> reset min/max measured values (resource ID 5605)
"""

rule_template_3: RuleDescriptor = create_compression_rule_template(
    template=all_templates[3], 
    rule_ID=b"\x03",
    mapping_indexes=[],
    msb_lsb_indexes=[16], 
    debug=False
)

### Template 4

In [9]:
""" 
Create compression rule for template 4

Description :
 - 2001:db8::3 -> 2001:db8::20
 - ACK (with CoAP Token, CoAP code 2.04)
"""

rule_template_4: RuleDescriptor = create_compression_rule_template(
    template=all_templates[4], 
    rule_ID=b"\x04", 
    msb_lsb_indexes=[16, 17], 
    debug=False
)

### Template 5

In [10]:
""" 
Create compression rule for template 5

Description :
 - 2001:db8::3 -> 2001:db8::20
 - ACK (with CoAP Token, CoAP code 2.04)
"""

rule_template_5: RuleDescriptor = create_compression_rule_template(
    template=all_templates[5], 
    msb_lsb_indexes=[16, 17], 
    rule_ID=b"\x05", 
    debug=False
)

### SCHC Context

In [11]:
from tools.context import ContextStatistics, evaluate, context_statistics_as_ascii_table

from microschc.rfc8724 import RuleNature
from microschc.rfc8724extras import Context
from microschc.manager import MatchStrategy

no_compression_rule_descriptor: RuleDescriptor = RuleDescriptor(id=Buffer(content=b'\x06', length=4), nature=RuleNature.NO_COMPRESSION)

rule_descriptors: List[RuleDescriptor] = [
    rule_template_0,
    rule_template_1,
    rule_template_2,
    rule_template_3,
    rule_template_4,
    rule_template_5,
    no_compression_rule_descriptor
]

schc_context: Context = Context(
    id='Orange',
    description='Context corresponding to leshan-thermostat-readings.pcapng dataset.',
    interface_id='none',
    ruleset=rule_descriptors,
    parser_id=Stack.IPV6_UDP_COAP
)
print(schc_context)

statistics: ContextStatistics = evaluate(context=schc_context, templates=all_templates, packets=all_packets, match_strategy=MatchStrategy.BEST)
print(context_statistics_as_ascii_table(statistics))

id:Orange description:Context corresponding to leshan-thermostat-readings.pcapng dataset. interface_id: none parser_id: IPv6-UDP-CoAP rules: 7
+-----+----------+-------+--------+---------+--------------+----------------+---------+---------+
| ID  | template | hits  |   CF   |  CFOC   | compressible | incompressible |  total  | residue |
+=====+==========+=======+========+=========+==============+================+=========+=========+
| 0   | 0        | 8543  | 4.074  | 8.887   | 4146168      | 730632         | 4876800 | 1197153 |
+-----+----------+-------+--------+---------+--------------+----------------+---------+---------+
| 1   | 1        | 592   | 10.537 | 10.537  | 255744       | 0              | 255744  | 24272   |
+-----+----------+-------+--------+---------+--------------+----------------+---------+---------+
| 2   | 2        | 380   | 14.270 | 14.270  | 200640       | 0              | 200640  | 14060   |
+-----+----------+-------+--------+---------+--------------+-------------